# 0. Imports and Data Load

In [1]:
import pandas as pd
import classifiers
import importlib
from igraph import Graph
import igraph as ig
import numpy as np
from ast import literal_eval
import concurrent.futures
import matplotlib.pyplot as plt
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
reactome_hypergeometric = pd.read_csv("../../data/processed/metrics/process_hypergeometric_apid_huri.csv", sep=',', header=0, index_col=0)
reactome_closeness = pd.read_csv("../../data/processed/metrics/process_closeness_apid_huri.csv", sep=',', header=0, index_col=0)
reactome_betweenness = pd.read_csv("../../data/processed/metrics/process_betweenness_apid_huri.csv", sep=',', header=0, index_col=0)
reactome_rwr = pd.read_csv("../../data/processed/metrics/process_rwr_apid_huri.csv", sep=',', header=0, index_col=0)
reactome_fraction_betweenness= pd.read_csv("../../data/processed/metrics/process_fraction_betweenness_apid_huri.csv", sep=',', header=0, index_col=0)

reactome_hypergeometric_80 = pd.read_csv("../../data/processed/metrics/process_ppi80_hyper_apid_huri.csv", sep=',', header=0, index_col=0)
reactome_closeness_80 = pd.read_csv("../../data/processed/metrics/process_ppi80_closeness_apid_huri.csv", sep=',', header=0, index_col=0)
reactome_betweenness_80 = pd.read_csv("../../data/processed/metrics/process_ppi80_betweenness_apid_huri.csv", sep=',', header=0, index_col=0)
reactome_rwr_80 = pd.read_csv("../../data/processed/metrics/process_ppi80_rwr_apid_huri.csv", sep=',', header=0, index_col=0)
reactome_fraction_betweenness_80 = pd.read_csv("../../data/processed/metrics/process_ppi80_fraction_betweenness_apid_huri.csv", sep=',', header=0, index_col=0)

reactome_hypergeometric_protein80 = pd.read_csv("../../data/processed/metrics/process_protein80_hyper_apid_huri.csv", sep=',', header=0, index_col=0)
reactome_closeness_protein80 = pd.read_csv("../../data/processed/metrics/process_protein80_closeness_apid_huri.csv", sep=',', header=0, index_col=0)
reactome_betweenness_protein80 = pd.read_csv("../../data/processed/metrics/process_protein80_betweenness_apid_huri.csv", sep=',', header=0, index_col=0)
reactome_rwr_protein80 = pd.read_csv("../../data/processed/metrics/process_protein80_rwr_apid_huri.csv", sep=',', header=0, index_col=0)
reactome_fraction_betweenness_protein80 = pd.read_csv("../../data/processed/metrics/process_protein80_fraction_betweenness_apid_huri.csv", sep=',', header=0, index_col=0)

In [4]:
graph = Graph.Read_GML("../../data/processed/graph_apid_huri")
reactome_proteins_indexes_df = pd.read_csv("../../data/processed//reactome_proteins_indexes_apid_huri.csv", sep=',', header=0)
reactome_neglogp_ar = np.array(reactome_hypergeometric)

reactome_proteins_indexes_df['protein_index'] = reactome_proteins_indexes_df['protein_index'].apply(literal_eval)
reactome_proteins_indexes_df['proteins_ids'] = reactome_proteins_indexes_df['proteins_ids'].apply(literal_eval)

In [ ]:
reactome_labels = np.zeros(reactome_hypergeometric.shape)
column = 0
for indexes in reactome_proteins_indexes_df['protein_index'].values:
    for pos in indexes:
        reactome_labels[pos][column] = 1
    column += 1
np.savetxt("../../data/processed/reactome_labels_apid_huri.csv", reactome_labels, delimiter=",")

In [5]:
reactome_labels_df = pd.read_csv("../../data/processed/reactome_labels_apid_huri.csv", sep=',', names=reactome_proteins_indexes_df['process'].values)
reactome_labels_df['protein_id'] = graph.vs['name']
reactome_labels_df.set_index('protein_id', inplace=True)

In [11]:
ppi80_hyper_list = []
ppi80_closeness_list = []
ppi80_betweenness_list = []
ppi80_fraction_betweenness_list = []
ppi80_rwr_list = []
label_list = []

for i, df in reactome_hypergeometric_80.groupby(level=0):
    df.set_index('level_1', inplace=True)
    ppi80_hyper_list.append(df)
    label_list.append(reactome_labels_df[reactome_labels_df.index.isin(df.index)])

for i, df in reactome_closeness_80.groupby(level=0):
    df.set_index('level_1', inplace=True)
    ppi80_closeness_list.append(df)
    
for i, df in reactome_betweenness_80.groupby(level=0):
    df.set_index('level_1', inplace=True)
    ppi80_betweenness_list.append(df)
    
for i, df in reactome_rwr_80.groupby(level=0):
    df.set_index('level_1', inplace=True)
    ppi80_rwr_list.append(df)
    
for i, df in reactome_fraction_betweenness_80.groupby(level=0):
    df.set_index('index', inplace=True)
    ppi80_fraction_betweenness_list.append(df)
    
protein80_hyper_list = []
protein80_closeness_list = []
protein80_betweenness_list = []
protein80_rwr_list = []
protein80_fraction_betweenness_list = []
protein_label_list = []

for i, df in reactome_hypergeometric_protein80.groupby(level=0):
    df.set_index('level_1', inplace=True)
    protein80_hyper_list.append(df)
    protein_label_list.append(reactome_labels_df[reactome_labels_df.index.isin(df.index)])
    
for i, df in reactome_closeness_protein80.groupby(level=0):
    df.set_index('level_1', inplace=True)
    protein80_closeness_list.append(df)
    
for i, df in reactome_betweenness_protein80.groupby(level=0):
    df.set_index('level_1', inplace=True)
    protein80_betweenness_list.append(df)
    
for i, df in reactome_rwr_protein80.groupby(level=0):
    df.set_index('level_1', inplace=True)
    protein80_rwr_list.append(df)
    
for i, df in reactome_fraction_betweenness_protein80.groupby(level=0):
    df.set_index('index', inplace=True)
    protein80_fraction_betweenness_list.append(df)

In [7]:
reactome_hypergeometric_test_indices = pd.read_csv("../../data/processed/fs/reactome_hyper_test_apid_huri.csv", sep=',', header=0, dtype='Int64', names=reactome_hypergeometric.columns)-1
reactome_closeness_test_indices = pd.read_csv("../../data/processed/fs/reactome_closeness_test_apid_huri.csv", sep=',', header=0, dtype='Int64', names=reactome_closeness.columns)-1
reactome_betweenness_test_indices = pd.read_csv("../../data/processed/fs/reactome_betweenness_test_apid_huri.csv", sep=',', header=0, dtype='Int64', names=reactome_betweenness.columns)-1
reactome_rwr_test_indices = pd.read_csv("../../data/processed/fs/reactome_rwr_test_apid_huri.csv", sep=',', header=0, dtype='Int64', names=reactome_rwr.columns)-1
reactome_fraction_betweenness_test_indices = pd.read_csv("../../data/processed/fs/reactome_fraction_betweenness_test_apid_huri.csv", sep=',', header=0, dtype='Int64', names=reactome_fraction_betweenness.columns)-1

reactome_hypergeometric_80_test_index = np.array(np.array_split(pd.read_csv("../../data/processed/fs/reactome_hyper_80_test_apid_huri.csv", sep=',', header=0).transpose(), 10, axis = 1)).astype(int) -1
reactome_closeness_80_test_index = np.array(np.array_split(pd.read_csv("../../data/processed/fs/reactome_closeness_80_test_apid_huri.csv", sep=',', header=0).transpose(), 10, axis = 1)).astype(int) -1
reactome_betweenness_80_test_index = np.array(np.array_split(pd.read_csv("../../data/processed/fs/reactome_betweenness_80_test_apid_huri.csv", sep=',', header=0).transpose(), 10, axis = 1)).astype(int) -1
reactome_rwr_80_test_index = np.array(np.array_split(pd.read_csv("../../data/processed/fs/reactome_rwr_80_test_apid_huri.csv", sep=',', header=0).transpose(), 10, axis = 1)).astype(int) -1
reactome_fraction_betweenness_80_test_index = np.array(np.array_split(pd.read_csv("../../data/processed/fs/reactome_fraction_betweenness_80_test_apid_huri.csv", sep=',', header=0).transpose(), 10, axis = 1)).astype(int) -1

reactome_hypergeometric_protein80_test_index = np.array(np.array_split(pd.read_csv("../../data/processed/fs/reactome_hyper_protein80_test_apid_huri.csv", sep=',', header=0).transpose(), 10, axis = 1)).astype(int) -1
reactome_closeness_protein80_test_index = np.array(np.array_split(pd.read_csv("../../data/processed/fs/reactome_closeness_protein80_test_apid_huri.csv", sep=',', header=0).transpose(), 10, axis = 1)).astype(int) -1
reactome_betweenness_protein80_test_index = np.array(np.array_split(pd.read_csv("../../data/processed/fs/reactome_betweenness_protein80_test_apid_huri.csv", sep=',', header=0).transpose(), 10, axis = 1)).astype(int) -1
reactome_rwr_protein80_test_index = np.array(np.array_split(pd.read_csv("../../data/processed/fs/reactome_rwr_protein80_test_apid_huri.csv", sep=',', header=0).transpose(), 10, axis = 1)).astype(int) -1
reactome_fraction_betweenness_protein80_test_index = np.array(np.array_split(pd.read_csv("../../data/processed/fs/reactome_fraction_betweenness_protein80_test_apid_huri.csv", sep=',', header=0).transpose(), 10, axis = 1)).astype(int) -1

# 1. Threshold Classifier - Complete Network

In [39]:
importlib.reload(classifiers)
hypergeometric_simple_clf = classifiers.simple_classifier(reactome_hypergeometric, reactome_labels_df, reactome_hypergeometric_test_indices, op_metric='fmeasure')
hypergeometric_simple_clf.to_csv('../../models/apid_huri/process_hypergeometric_simple.csv', index=False)

  0%|          | 0/429 [00:00<?, ?it/s]

Process R-HSA-1031716 CV results:
Train F-measure: 0.286 +/- 0.032
Validation F-measure: 0.284 +/- 0.290
Test F-measure: 0.400
Precision@20: 0.25
Precision@15: 0.3333333333333333
Precision@14: 0.35714285714285715

Process R-HSA-112379 CV results:
Train F-measure: 0.581 +/- 0.020
Validation F-measure: 0.509 +/- 0.192
Test F-measure: 0.750
Precision@20: 0.5
Precision@15: 0.6666666666666666
Precision@11: 0.6363636363636364

Process R-HSA-112385 CV results:
Train F-measure: 0.640 +/- 0.021
Validation F-measure: 0.582 +/- 0.176
Test F-measure: 0.560
Precision@20: 0.4
Precision@15: 0.4666666666666667
Precision@11: 0.6363636363636364

Process R-HSA-1168640 CV results:
Train F-measure: 0.777 +/- 0.038
Validation F-measure: 0.675 +/- 0.468
Test F-measure: 0.889
Precision@20: 0.5
Precision@15: 0.6666666666666666
Precision@10: 0.8

Process R-HSA-1214188 CV results:
Train F-measure: 0.223 +/- 0.023
Validation F-measure: 0.170 +/- 0.236
Test F-measure: 0.300
Precision@20: 0.15
Precision@15: 0.2
Pre

In [42]:
importlib.reload(classifiers)
closeness_simple_clf = classifiers.simple_classifier(reactome_closeness, reactome_labels_df, reactome_closeness_test_indices, op_metric='fmeasure')
closeness_simple_clf.to_csv('../../models/apid_huri/process_closeness_simple.csv', index=False)

  0%|          | 0/429 [00:00<?, ?it/s]

Process R-HSA-1031716 CV results:
Train F-measure: 0.058 +/- 0.005
Validation F-measure: 0.017 +/- 0.031
Test F-measure: 0.000
Precision@20: 0.0
Precision@15: 0.0
Precision@14: 0.0

Process R-HSA-112379 CV results:
Train F-measure: 0.615 +/- 0.026
Validation F-measure: 0.591 +/- 0.276
Test F-measure: 0.375
Precision@20: 0.2
Precision@15: 0.2
Precision@11: 0.2727272727272727

Process R-HSA-112385 CV results:
Train F-measure: 0.522 +/- 0.024
Validation F-measure: 0.474 +/- 0.225
Test F-measure: 0.700
Precision@20: 0.45
Precision@15: 0.5333333333333333
Precision@11: 0.6363636363636364

Process R-HSA-1168640 CV results:
Train F-measure: 0.795 +/- 0.029
Validation F-measure: 0.714 +/- 0.272
Test F-measure: 0.737
Precision@20: 0.4
Precision@15: 0.5333333333333333
Precision@10: 0.7

Process R-HSA-1214188 CV results:
Train F-measure: 0.211 +/- 0.022
Validation F-measure: 0.205 +/- 0.198
Test F-measure: 0.235
Precision@20: 0.15
Precision@15: 0.2
Precision@11: 0.2727272727272727

Process R-HSA-1

In [8]:
importlib.reload(classifiers)
betweenness_simple_clf = classifiers.simple_classifier(reactome_betweenness, reactome_labels_df, reactome_betweenness_test_indices, op_metric='fmeasure')
betweenness_simple_clf.to_csv('../../models/apid_huri/process_betweenness_simple.csv', index=False)

  0%|          | 0/429 [00:00<?, ?it/s]

Process R-HSA-1031716 CV results:
Train F-measure: 0.010 +/- 0.000
Validation F-measure: 0.009 +/- 0.000
Test F-measure: 0.000
Precision@20: 0.0
Precision@15: 0.0
Precision@14: 0.0

Process R-HSA-112379 CV results:
Train F-measure: 0.012 +/- 0.004
Validation F-measure: 0.005 +/- 0.003
Test F-measure: 0.010
Precision@20: 0.0
Precision@15: 0.0
Precision@11: 0.0

Process R-HSA-112385 CV results:
Train F-measure: 0.014 +/- 0.004
Validation F-measure: 0.011 +/- 0.030
Test F-measure: 0.000
Precision@20: 0.0
Precision@15: 0.0
Precision@11: 0.0

Process R-HSA-1168640 CV results:
Train F-measure: 0.045 +/- 0.030
Validation F-measure: 0.006 +/- 0.000
Test F-measure: 0.000
Precision@20: 0.0
Precision@15: 0.0
Precision@10: 0.0

Process R-HSA-1214188 CV results:
Train F-measure: 0.008 +/- 0.001
Validation F-measure: 0.003 +/- 0.003
Test F-measure: 0.000
Precision@20: 0.0
Precision@15: 0.0
Precision@11: 0.0

Process R-HSA-1234159 CV results:
Train F-measure: 0.028 +/- 0.008
Validation F-measure: 0.0

In [9]:
importlib.reload(classifiers)
rwr_simple_clf = classifiers.simple_classifier(reactome_rwr, reactome_labels_df, reactome_rwr_test_indices, op_metric='fmeasure')
rwr_simple_clf.to_csv('../../models/apid_huri/process_rwr_simple.csv', index=False)

  0%|          | 0/429 [00:00<?, ?it/s]

Process R-HSA-1031716 CV results:
Train F-measure: 1.000 +/- 0.000
Validation F-measure: 0.991 +/- 0.029
Test F-measure: 0.963
Precision@20: 0.7
Precision@15: 0.9333333333333333
Precision@14: 1.0

Process R-HSA-112379 CV results:
Train F-measure: 1.000 +/- 0.000
Validation F-measure: 0.989 +/- 0.035
Test F-measure: 0.952
Precision@20: 0.55
Precision@15: 0.7333333333333333
Precision@11: 1.0

Process R-HSA-112385 CV results:
Train F-measure: 1.000 +/- 0.000
Validation F-measure: 0.989 +/- 0.035
Test F-measure: 1.000
Precision@20: 0.55
Precision@15: 0.7333333333333333
Precision@11: 1.0

Process R-HSA-1168640 CV results:
Train F-measure: 1.000 +/- 0.000
Validation F-measure: 0.989 +/- 0.035
Test F-measure: 1.000
Precision@20: 0.5
Precision@15: 0.6666666666666666
Precision@10: 1.0

Process R-HSA-1214188 CV results:
Train F-measure: 0.939 +/- 0.010
Validation F-measure: 0.936 +/- 0.086
Test F-measure: 1.000
Precision@20: 0.55
Precision@15: 0.7333333333333333
Precision@11: 1.0

Process R-HSA-

In [25]:
importlib.reload(classifiers)
fraction_betweenness_simple_clf = classifiers.simple_classifier(reactome_fraction_betweenness, reactome_labels_df, reactome_betweenness_test_indices, op_metric='fmeasure')
fraction_betweenness_simple_clf.to_csv('../../models/apid_huri/process_fraction_betweenness_simple.csv', index=False)

  0%|          | 0/429 [00:00<?, ?it/s]

Process R-HSA-1031716 CV results:
Train F-measure: 0.166 +/- 0.014
Validation F-measure: 0.157 +/- 0.131
Test F-measure: 0.111
Precision@20: 0.1
Precision@15: 0.13333333333333333
Precision@14: 0.14285714285714285

Process R-HSA-112379 CV results:
Train F-measure: 0.411 +/- 0.023
Validation F-measure: 0.246 +/- 0.221
Test F-measure: 0.455
Precision@20: 0.25
Precision@15: 0.3333333333333333
Precision@11: 0.45454545454545453

Process R-HSA-112385 CV results:
Train F-measure: 0.472 +/- 0.024
Validation F-measure: 0.371 +/- 0.201
Test F-measure: 0.100
Precision@20: 0.2
Precision@15: 0.2
Precision@11: 0.09090909090909091

Process R-HSA-1168640 CV results:
Train F-measure: 0.646 +/- 0.020
Validation F-measure: 0.636 +/- 0.206
Test F-measure: 0.609
Precision@20: 0.4
Precision@15: 0.4666666666666667
Precision@10: 0.6

Process R-HSA-1214188 CV results:
Train F-measure: 0.233 +/- 0.027
Validation F-measure: 0.176 +/- 0.216
Test F-measure: 0.087
Precision@20: 0.1
Precision@15: 0.06666666666666667


# 2. Threshold Classifier - Reduced Networks

## 2.1. PPI 80% Reduction

In [ ]:
importlib.reload(classifiers)
hyper80_simple_clf = classifiers.reduced_classifiers(2, None, None, ppi80_hyper_list, reactome_hypergeometric_80_test_index, None, label_list)
hyper80_simple_clf.to_csv('../../models/apid_huri/process_hyper80_simple.csv', index=False)

In [ ]:
importlib.reload(classifiers)
rwr80_simple_clf = classifiers.reduced_classifiers(2, None, None, ppi80_rwr_list, reactome_rwr_80_test_index, None, label_list)
rwr80_simple_clf.to_csv('../../models/apid_huri/process_rwr80_simple.csv', index=False)

In [9]:
importlib.reload(classifiers)
closeness80_simple_clf = classifiers.reduced_classifiers(2, None, None, ppi80_closeness_list, reactome_closeness_80_test_index, None, label_list)
closeness80_simple_clf.to_csv('../../models/apid_huri/process_closeness80_simple.csv', index=False)

  0%|          | 0/10 [00:00<?, ?it/s]

IndexError: list index out of range

In [26]:
importlib.reload(classifiers)
betweenness80_simple_clf = classifiers.reduced_classifiers(2, None, None, ppi80_betweenness_list, reactome_betweenness_80_test_index, None, label_list)
betweenness80_simple_clf.to_csv('../../models/apid_huri/process_betweenness80_simple.csv', index=False)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

...

  0%|          | 0/429 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

In [48]:
importlib.reload(classifiers)
fraction_betweenness80_simple_clf = classifiers.reduced_classifiers(2, None, None, ppi80_fraction_betweenness_list, reactome_fraction_betweenness_80_test_index, None, label_list)
fraction_betweenness80_simple_clf.to_csv('../../models/apid_huri/process_fraction_betweenness80_simple.csv', index=False)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

...

  0%|          | 0/429 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

## 2.2. Protein 80% Reduction

In [9]:
importlib.reload(classifiers)
hyper_protein80_simple_clf = classifiers.reduced_classifiers(2, None, None, protein80_hyper_list, reactome_hypergeometric_protein80_test_index, None, protein_label_list)
hyper_protein80_simple_clf.to_csv('../../models/apid_huri/process_hyper_protein80_simple.csv', index=False)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

...

  0%|          | 0/429 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

In [10]:
importlib.reload(classifiers)
rwr_protein80_simple_clf = classifiers.reduced_classifiers(2, None, None, protein80_rwr_list, reactome_rwr_protein80_test_index, None, protein_label_list)
rwr_protein80_simple_clf.to_csv('../../models/apid_huri/process_rwr_protein80_simple.csv', index=False)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

...

  0%|          | 0/429 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

In [11]:
importlib.reload(classifiers)
closeness_protein80_simple_clf = classifiers.reduced_classifiers(2, None, None, protein80_closeness_list, reactome_closeness_protein80_test_index, None, protein_label_list)
closeness_protein80_simple_clf.to_csv('../../models/apid_huri/process_closeness_protein80_simple.csv', index=False)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

...

  0%|          | 0/429 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

In [12]:
importlib.reload(classifiers)
betweenness_protein80_simple_clf = classifiers.reduced_classifiers(2, None, None, protein80_betweenness_list, reactome_betweenness_protein80_test_index, None, protein_label_list)
betweenness_protein80_simple_clf.to_csv('../../models/apid_huri/process_betweenness_protein80_simple.csv', index=False)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

...

  0%|          | 0/429 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

In [13]:
importlib.reload(classifiers)
fraction_betweenness_protein80_simple_clf = classifiers.reduced_classifiers(2, None, None, protein80_fraction_betweenness_list, reactome_fraction_betweenness_protein80_test_index, None, protein_label_list)
fraction_betweenness_protein80_simple_clf.to_csv('../../models/apid_huri/process_fraction_betweenness_protein80_simple.csv', index=False)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

...

  0%|          | 0/429 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]

  0%|          | 0/429 [00:00<?, ?it/s]